In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
from dateutil import rrule
import datetime as dt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import mpld3
import geojson
%matplotlib inline


In [2]:
annual_earthquakes = pickle.load( open( "earthquake_Jan2016.p", "rb" ) )

In [3]:
cnt = 0
proplist = list()
keys = list(annual_earthquakes[0].properties.keys())
keys.append('long')
keys.append('lat')
proplist.append(keys)
for indx, quake in enumerate(annual_earthquakes):
    values = list(quake.properties.values())
    values.append(quake.geometry.coordinates[0])
    values.append(quake.geometry.coordinates[1])
    proplist.append(values)
np_proplist = np.array(proplist).astype(np.str)

df = pd.DataFrame(np_proplist[1:],columns=list(np_proplist[0]))
df = df.set_index('time')
# df.index = (df.index/1000).astype(np.int64)
df.index = pd.to_datetime(df.index ,unit='ms')
df = df.convert_objects(convert_numeric=True)
df = df.fillna(0)
df = df.sort_index()

C:\Python35\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [11]:

# https://mpld3.github.io/examples/scatter_tooltip.html
mpld3.enable_notebook()
fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'))

# s=1000 * np.random.random(size=N),
                     
mag_np = df[['mag']].ix[0:,0].as_matrix()
sig_np = df[['sig']].ix[0:,0].as_matrix()
time_np = np.array(df.index.astype(np.str))
# df[['time']].ix[0:,0].as_matrix()
N = len(sig_np)


scatter = ax.scatter(mag_np,
                     sig_np,
                     c=np.random.random(size=N),
                     alpha=0.3,
                     cmap=plt.cm.jet)
ax.grid(color='white', linestyle='solid')
ax.set_autoscale_on(False)

ax.set_title("Magnitude(mag) vs Significance(sig)- Time as tooltip", size=15)
ax.set_xlabel("Magnitude in Richter scale")
ax.set_ylabel("Significance")
labels = [str(time_np[i]) for i in range(N)]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
mpld3.plugins.connect(fig, tooltip)

mpld3.display()

In [17]:
# Sonic Boom
# df[df.index == '2016-01-28 18:23:45.000' ][['title']]

# Intra Slab Event
df[df.index == '2016-01-03 23:05:22.270' ][['title']]

df[df.index == '2016-01-24 10:30:30.230' ][['title']]

,title
time,
2016-01-24 10:30:30.230,"M 7.1 - 86km E of Old Iliamna, Alaska"


In [7]:
# Sonic Boom Outlier - M 0.0 Sonic Boom - 12km SE of Atlantic City, N
# http://foxtrotalpha.jalopnik.com/mystery-sonic-boom-did-not-come-from-atlantic-city-base-1755786208
# http://www.disaster-report.com/2016/01/sonic-boom-new-jersey-earthquake.html